<a href="https://colab.research.google.com/github/SERVIR/flood_mapping_intercomparison/blob/version_2/notebooks/Module_3_HYDRAFloods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipyleaflet==0.18.2 geemap hydrafloods

INFO: pip is looking at multiple versions of geemap to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.6/86.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.3 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=16eae750294ac341bfb9a3e1add672ddadc457107350dd9e4786612eb859da9e
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e1

Now that we have installed our packages, it's time to import them via `import` so we can use them in this notebook. Below you can find a brief description of what each package does.

* The `geemap` package will allow us to visualize a Google Maps interface interactively within this notebook.
* The `ee` package will allow us to run Earth Engine functions via the python programming language.

In [2]:
import hydrafloods as hf
import geemap
import geemap.colormaps as cm
import ee

Now, we have to authenticate and initialize earth engine. After you run the code below, click through the pop-up window to login to the Google Account associated with your Google Earth Engine account. Click "Continue" until you have returned to this notebook and a green checkmark appears to the left of the code cell below.

If the below cell does not run, try clicking runtime --> Disconnect and Delete Runtime, then click "Connect" in the upper right hand corner. Then rerun the first couple cells in the notebook. You may have to do this multiple times.

If you get an error when running the below cell that says "EEException, you do not have access to the project "rami-for-itc", you need to redo the steps in module 1 as you are not yet added to the cloud project.

In [3]:
ee.Authenticate()

ee.Initialize(project = 'servir-sco-assets')

In [ ]:
#parent_directory = "projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study"
parent_directory = "users/mickymags/flood_intercomparison_chad_09_26"
aoi = ee.FeatureCollection(parent_directory + 'aoi')

roi = aoi.geometry()

aoi_centroid = aoi.geometry().centroid()             # Get the center of the AOI
lon = aoi_centroid.coordinates().get(0).getInfo()    # Extract the longitude from the centroid
lat = aoi_centroid.coordinates().get(1).getInfo()    # Extract the latitude from the centroid

In [ ]:
#point1 = [14.8309, 10.905]
#point2 = [14.4221, 12.88]
#point3 = [15.1472, 12.7648]
#point4 = [15.8394, 11.212]

In [ ]:
#roi = ee.Geometry.Polygon([[point1, point2, point3, point4]])

In [ ]:
sd = "2023-06-16"          # start date
ed = "2023-06-17"          # end date

In [ ]:
from hydrafloods import corrections

In [ ]:
s1 = hf.Sentinel1(roi, sd, ed)

In [ ]:
_ = geemap.Map()

In [ ]:
hf.__version__

'2023.10.14'

In [ ]:
num_imgs = s1.n_images
print('Number of images in the dataset:', num_imgs)

Number of images in the dataset: 4


# Terrain Correction

In [ ]:
elv = ee.Image("JAXA/ALOS/AW3D30/V2_2").select("AVE_DSM")

s1_speckle = s1.apply_func(hf.gamma_map)

s1_tc = s1_speckle.apply_func(corrections.slope_correction, elevation=elv, buffer=30)

In [ ]:
s1_med = s1_tc.collection.median().clip(aoi)

In [ ]:
bmax = hf.bmax_otsu(
    s1_med,
    band = 'VV',
    region = roi,
    initial_threshold = -20,
    thresh_no_data = -16,
    scale = 300,
    bmax_threshold = 0.75
)

In [ ]:
edge = hf.edge_otsu(
    s1_med,
    band = 'VV',
    region = roi,
    edge_buffer = 300,
    initial_threshold = -20,
    thresh_no_data = -15,
    scale = 30
).clip(roi)

In [ ]:
water_viz = {
    "min": 0,
    "max": 1,
    "palette": ['D3D3D3', '000080']
}

s1_viz = {
    'bands': ['VV'],
    'min': -25,
    'max': 0
}

In [ ]:
edge_default = hf.edge_otsu(
    s1_med,
    band = 'VV',
    region = roi,
    #edge_buffer = 100,
    #initial_threshold = 0,
    thresh_no_data = -0.2,
    #scale = 90
).clip(roi)

In [ ]:
edge_low_thresh = hf.edge_otsu(
    s1_med,
    band = 'VV',
    region = roi,
    edge_buffer = 200,
    initial_threshold = -20,
    thresh_no_data = -0.2,
    #scale = 30
).clip(roi)

In [ ]:
edge_med_thresh = hf.edge_otsu(
    s1_med,
    band = 'VV',
    region = roi,
    edge_buffer = 200,
    initial_threshold = -15,
    thresh_no_data = -0.2,
    #scale = 30
).clip(roi)

In [ ]:
Map = geemap.Map(center = (lat, lon), zoom = 7)
#Map.addLayer(s1_med, s1_viz, 'Sentinel-1')
#Map.addLayer(edge_low_thresh, water_viz, 'Edge -20')
Map.addLayer(edge_med_thresh, water_viz, 'Edge -18')
Map.addLayerControl()
Map

Map(center=[31.244961096953492, 74.27411570164877], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# Define a function that exports an Image to a Google Earth Engine Asset.
def img_exporter(img, asset_id, scale_):

  desc = 'Flood_Map_Export_'
  region_ = aoi.geometry()
  geemap.ee_export_image_to_asset(image = img,
                                  assetId = asset_id,
                                  description = desc,
                                  region = region_,
                                  crs = 'EPSG:4326',
                                  scale = scale_,
                                  maxPixels = 1e13)
  return 0

In [ ]:
img_exporter(edge, 'hydrafloods_mosaic', 30)